## 1.归并排序附加题
给定一个数组arr，两个整数lower和upper，返回arr中有多少个子数组累加和在[lower,upper]范围

In [2]:
#https://leetcode.com/problems/count-of-range-sum/
# 暴力解
# 遍历所有子数组O(N2)，如果子数组和每次遍历O(N)，整体为O(N3)
# 子数组和通过两个前缀和计算，优化为O(N2)

# 归并思路:O(N*logN)
# ======================================================================
# 任何一个子数组，以x为结尾的子数组达标的有几个，所有x凑齐，即为答案。
# 0结尾: 0-0
# 1结尾: 0-1,1-1
# 2结尾: 0-2,1-2,2-2
# 求以x为结尾的子数组，落在[L,U]上，若[0,x]的累加和为S，则可以转化为找到从0开始、以y为结尾的[0,y]累加和在[S-U,S-L]上
# 例如 已知[0,17] = 100， 要求落在[10,40]上：转化为求[0,y]上的累加和落在[60,90]上。
# 因为y一定小于x！ 题意转化为 已知从0出发的累计和preSum数组，数组中的每一个x，求左侧有多少个y能达标(在[S-U,S-L]上)
# 左侧的范围要求[S-U,S-L]，随着右侧递增，同时增大不回退


def getPreSum(arr):
    preSum = [None] * len(arr)
    s = 0
    for i in range(len(arr)):
        s += arr[i]
        preSum[i] = s
    return preSum

def merge(arr, L, M, R, lower, upper):
    #arr保证最少俩数
    #L...M有序；M+1...R有序
    ans = 0
    windowL = L
    windowR = L
    # lower <= x-y <= upper
    # min_y <= y <= max_y
    for x in arr[M+1:R+1]:#右侧
        min_y = x - upper
        max_y = x - lower
        # 保障左侧窗口[windowL, windowR)
        # 当仅有一个数y时，
        # 当 y < min_y,            windowL右滑、windowR右滑，windowR-windowL=0
        # 当 max_y < y,            windowL不动，windowR不动，windowR-windowL=0
        # 当 min_y <= y <= max_y,  windowL不动，windowR右滑，windowR-windowL=1
        while windowR <= M and arr[windowR] <= max_y:
            windowR += 1
        while windowL <= M and arr[windowL] < min_y:
            windowL += 1
        ans += (windowR - windowL)
    #正常mergeSort过程
    index = 0
    helpArr = [None] * (R-L+1)
    P1 = L
    P2 = M+1
    while P1 <= M and P2 <= R:
        if arr[P1] <= arr[P2]:
            helpArr[index] = arr[P1]
            index += 1
            P1 += 1
        else:
            helpArr[index] = arr[P2]
            index += 1
            P2 += 1
    while P1 <= M: 
        helpArr[index] = arr[P1]
        index += 1
        P1 += 1
    while P2 <= R: 
        helpArr[index] = arr[P2]
        index += 1
        P2 += 1
    for i in range(R-L+1):
        arr[i+L] = helpArr[i]
    return ans

# arr[L...R]变为preSum;求在原始的arr[L...R]中，有多少个子数组累加和在[lower,upper]上
def count(preSum, L, R, lower, upper):
    #代表只有一个preSum(L)，则只判断这个数是否在[lower, upper]上即可
    if L == R:
        if preSum[L] >= lower and preSum[L] <= upper:
            return 1
        else:
            return 0
    #L!=R，上不止一个数
    mid = L + ((R-L)>>1)
    leftPart = count(preSum, L, mid, lower, upper)
    rightPart = count(preSum, mid+1, R, lower, upper)
    mergeRes = merge(preSum, L, mid, R, lower, upper)
    return leftPart+rightPart+mergeRes

def countRangeSum(nums, lower, upper):
    if not nums:
        return 0
    preSum = getPreSum(nums)
    return count(preSum, 0, len(nums)-1, lower, upper)

In [4]:
#leetcode solution：一样的方法
def countRangeSum(nums, lower, upper):
    first = [0]
    for num in nums:
        first.append(first[-1] + num)
    def sorting(lo, hi):
        mid = (lo + hi) // 2
        if mid == lo: #保障至少俩数
            return 0
        count = sorting(lo, mid) + sorting(mid, hi)
        i = j = mid #右侧第一个数
        for left in first[lo:mid]:
            while i < hi and first[i] - left <  lower: i += 1
            while j < hi and first[j] - left <= upper: j += 1
            count += j - i
        first[lo:hi] = sorted(first[lo:hi])
        return count
    return sorting(0, len(first))


## 2.快速排序

### 2.1 荷兰国旗问题O(N)
给定一个数x和数组arr，让数组的左侧<x，中间=x，右侧>x，返回中间的边界

In [5]:
#分为小于区，大于区内可以无序
# 当前数<目标，当前数与小于区下一个交换，小于区右扩，当前数跳下一个
# 当前数=目标，当前数跳下一个
# 当前数>目标，当前数与大于区前一个交换，大于区左扩，当前数不跳！

def swap(arr, i, j):
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp
    
def netherlandsFlag(arr, L, R):
    if L > R:
        return -1,-1
    if L == R:
        return L,R
    less = L -1
    more = R #直接把R放入大于区域
    index = L
    while index < more:
        if arr[index] == arr[R]:
            index += 1
        elif arr[index] < arr[R]:
            swap(arr, index, less+1)
            index += 1
            less += 1
        else:
            swap(arr, index, more-1)
            more -= 1
    swap(arr, more, R) #最后把大于区域的R取出来与大于区域的第一个数交换
    return less+1, more

### 2.2快速排序O(N*logN)

In [7]:
#以arr[N-1]做为荷兰国旗的x，做排序后，把arr[N-1]与大于区的前一个交换
import random

def process(arr, L, R):
    if L >= R:
        return 
    swap(arr, L+random.randint(0, R-L), R) #随机取一个数作为R
    equalL, equalR = netherlandsFlag(arr, L, R)
    process(arr, L, equalL - 1)
    process(arr, equalR+1, R)
    
def quickSort(arr):
    if not arr or len(arr) < 2:
        return
    process(arr, 0, len(arr)-1)
    return

In [8]:
#快排非递归版本
def quickSort2(arr):
    if not arr or len(arr) < 2:
        return
    N = len(arr) - 1
    jobs = list()
    jobs.append((0, N))
    while jobs:
        jobL, jobR = jobs.pop()
        if jobL < jobR: #有效
            swap(arr, jobL+random.randint(0, jobR-jobL), jobR)
            equalL, equalR = netherlandsFlag(arr, jobL, jobR)
            jobs.extends([(jobL, equalL-1), (equalR+1, jobR)])
    return
         